In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c dogs-vs-cats

 99% 808M/812M [00:03<00:00, 237MB/s]
100% 812M/812M [00:03<00:00, 236MB/s]


In [ ]:
!unzip -qq dogs-vs-cats.zip

In [ ]:
!unzip -qq test1.zip
!unzip -qq train.zip

In [ ]:
import os, shutil, pathlib

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")


In [ ]:
def make_subset(subset_name, start_index, end_index):
  for category in ("cat", "dog"):
    dir = new_base_dir / subset_name / category
    os.makedirs(dir)
    fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
    for fname in fnames:
      shutil.copyfile(src=original_dir / fname,dst=dir / fname)

In [ ]:
make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=["accuracy"])

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory
train_dataset = image_dataset_from_directory(
                                              new_base_dir / "train",
                                              image_size=(180, 180),
                                              batch_size=32)
validation_dataset = image_dataset_from_directory(
                                              new_base_dir / "validation",
                                              image_size=(180, 180),
                                              batch_size=32)
test_dataset = image_dataset_from_directory(
                                              new_base_dir / "test",
                                              image_size=(180, 180),
                                              batch_size=32)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [ ]:
callbacks = [
keras.callbacks.ModelCheckpoint(filepath="convnet_from_scratch.keras",save_best_only=True,monitor="val_loss")
]

In [ ]:
history = model.fit(train_dataset,epochs=10,validation_data=validation_dataset,callbacks=callbacks)

Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 233s 4s/step - accuracy: 0.5108 - loss: 0.9008 - val_accuracy: 0.5000 - val_loss: 0.7043
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 265s 4s/step - accuracy: 0.5083 - loss: 0.6951 - val_accuracy: 0.5020 - val_loss: 0.6927
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 240s 3s/step - accuracy: 0.5306 - loss: 0.6949 - val_accuracy: 0.5340 - val_loss: 0.6769
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 262s 3s/step - accuracy: 0.5922 - loss: 0.6715 - val_accuracy: 0.6460 - val_loss: 0.6169
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 274s 4s/step - accuracy: 0.6323 - loss: 0.6356 - val_accuracy: 0.5860 - val_loss: 0.7020
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 249s 3s/step - accuracy: 0.6766 - loss: 0.6239 - val_accuracy: 0.6520 - val_loss: 0.6364
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 262s 3s/step - accuracy: 0.7183 - loss: 0.5549 - val_accuracy: 0.6750 - val_loss: 0.5863
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 276s 4s/step - accuracy: 0.7445 - loss: 0.5288 - val_accuracy: 0.7290 - v